In [ ]:
import spectre
import spectre.IO.H5 as spectre_h5
from spectre.Visualization.ReadH5 import to_dataframe
from spectre.Visualization.PlotCce import plot_cce

import scri
import spherical_functions as sf

import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Cauchy-Characteristic Evolution

In this tutorial we will cover how to obtain, run, and examine the output from SpECTRE's CCE module.

Most of the information in this tutorial can also be found in our [running CCE tutorial](https://spectre-code.org/tutorial_cce.html).

## Obtaining SpECTRE's CCE Module
 By far the easiest way to obtain the CCE module, is by downloading the pre-built binary from the latest [SpECTRE release](https://github.com/sxs-collaboration/spectre/releases).



In [ ]:
%%bash
mkdir -p runs
rm -rf runs/CceExecutables*
cd runs
wget https://github.com/sxs-collaboration/spectre/releases/download/v2024.08.03/CceExecutables.tar.xz
tar -xf CceExecutables.tar.xz

This tarball provides:

- the CCE executable `CharacteristicExtract`
- example worldtube data
- example input file
- example output from CCE
- worldtube data format converter
- python script to check example data

Before we do anything else, let's make sure the executable runs correctly and that we obtain the expected output. This should run for only a couple minutes.

In [ ]:
%%bash
cd runs/CceExecutables/
rm -f CharacteristicExtractReduction.h5
./CharacteristicExtract --input-file CharacteristicExtract.yaml
python CheckCceOutput.py

### SUCCESS: The CCE output is as expected! Yay!

If you get this message, awesome! That means the executable worked correctly.

## The input file

Now let's take a look the input .yaml file that we just used and study the settings that were used. This input file can already be used in its current form to produce production-level CCE waveforms. All you need to change is the path to the worldtube data.

Open the `runs/CceExecutables/CharacteristicExtract.yaml` file in a new tab to find some nice syntax highlighting. We'll briefly explain the varous options (i.e., which ones are important to change for every run and which ones you should probably not change).

## The worldtube data

Now let's take a look at what's required in the worldtube data, and how we expect it to be formatted. First there is an "alphabet soup" of Bondi variables that must be contained in the H5 worldtube file. All of these variables must be in the spectre H5::Dat format.

In [ ]:
%%bash
cd runs/CceExecutables/
h5ls -r BondiSachsCceR0200.h5

The exact meaning of these variables can be found in [arXiv:2110.08635](https://arxiv.org/abs/2110.08635).

In [ ]:
worldtube_data = spectre_h5.H5File("runs/CceExecutables/BondiSachsCceR0200.h5", "r")
bondi_J_dat_file = worldtube_data.get_dat("J")
print(f"Dimensions: {bondi_J_dat_file.get_dimensions()}")
print(f"Version: {bondi_J_dat_file.get_version()}")
print(f"Header:\n{bondi_J_dat_file.get_header()}")

# The legend is a bit too big to print, but we can calculate l_max from the size
legend = bondi_J_dat_file.get_legend()
print(legend[1])
l_max = int(np.sqrt((bondi_J_dat_file.get_dimensions()[1] - 1) / 2) - 1)
print(f"L max: {l_max}")

In [ ]:
# Make the dat file into a Pandas DataFrame for easy manipulation
bondi_J_df = to_dataframe(bondi_J_dat_file)

coef_to_plot = "Re(2,2)"


plt.plot(bondi_J_df["Time"], bondi_J_df[coef_to_plot])
plt.xlabel("Time (M)")
plt.ylabel(f"Bondi J {coef_to_plot}")

So this example data isn't that long. It's only to test out that the executable works.

## Real Worldtube Data

Let's take a look at some real worltube data for a BBH simulation run with SpECTRE.

In [ ]:
cce_data_dir = Path("/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce")
worldtube_r292_file = cce_data_dir / "BondiCceR0292.h5"
worldtube_data = spectre_h5.H5File(worldtube_r292_file, "r")

# Now it's your turn!
# - Open one of the bondi variable dat files from the H5 file
# - Convert it to a pandas DataFrame
# - Choose a coefficient and plot it!

## Running CCE

First, let's edit the input file in the `runs/CceExecutables` directory to point to the full worltube data. This would be the `BoundaryDataFilename` option. Change it to now point to `/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/BondiSachsCceR0150.h5`

Then, in the `03_Wednesday` directory there is a submit script for you to edit. The only thing you should have to add is the actual command that runs the executable.

> Hint: To specify the input file to run with, add `--input-file NameOfInputFile.yaml` when you run the executable

Once you've done this, submit your job!

In [ ]:
%%bash
sbatch SubmitCCE.sh

This run will take several hours to finish, so you'll have to wait a bit to get your own data.

But don't worry! We've already run CCE for you so have a chance to look at the output data.

## Real CCE output data

The CCE output data is located at `/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/CharacteristicExtractReduction.h5`.

Our spectre H5 python bindings know how to read the CCE output data

In [ ]:
cce_output_file_name = cce_data_dir / "CharacteristicExtractReduction.h5"
cce_output_file = spectre_h5.H5File(cce_output_file_name, "r")
# The subfile name always has the extraction radius in it
cce_output = cce_output_file.get_cce("SpectreR0292", 8)

# get_dimension requires you to specify a Bondi variable.
# The choices are 'Strain', 'News', 'Psi0', ..., 'Psi4'
print(f"Dimensions Psi0: {cce_output.get_dimensions('Psi0')}")
print(f"Version: {cce_output.get_version()}")
print(f"Header:\n{cce_output.get_header()}")
legend = cce_output.get_legend()
print(f"Legend size: {len(legend)}")

Rather than loading the data yourself and plotting it, the SpECTRE CLI and PyBindings conveniently provide a function that plots the interesting CCE output quantities. This includes

- Strain
- News
- All Weyl scalars

On the command line, to execute this function you can run

```sh
spectre plot cce -h
```

but since we're in a notebook, we can just call the function directly. This won't provide you with paper-quality plots of these quantities, but it can be extremely useful to obtain an initial idea of what your output looks like.

Pick your favorite spherical harmonic modes to plot! Which ones from the previous session seem like they'd be interesting?

In [ ]:
# Modes must be in the format: "Real Y_l,m" or "Imag Y_l,m"
# Also, you must currently specify at least two otherwise you'll get an error.
modes = ["Real Y_2,2", "Imag Y_2,2"]
fig = plot_cce(cce_output_file_name, modes)
plt.show()

# Working with `scri`

## Creating an AsymptoticBondiData (ABD) object using `scri`

Now that we have an understanding of how to run SpECTRE's CCE module, we'll look into the python package `scri` which contains a number of useful tools for studying and working with the outputs of CCE.

We'll start by construcing an `AsymptoticBondiData` (`ABD`) object, which contains
- the shear $\sigma=\bar{h}/2$,
- and the Weyl scalars $\Psi_{i}$ for $i\in\{0,1,2,3,4\}$.

Each of these objects are stored as `ModesTimeSeries` objects, which is simply a class containing the spin-weight, time, and modal data of each object.

To create an `ABD` object, one can simply use the `scri.create_abd_from_h5` function! All this needs is
- `file_format`: the file format (which for CCE outputs is `SpECTRECCE_v1')
- `file_name`: the CCE output file name

But some other useful optional arguments are:
- `radius`: the radius of the worldtube input to CCE (if this is set, the time array of the `ABD` object is time translated by the radius)
- `ch_mass`: the total Christodoulou mass of the system (if this is set, the mass is scaled out of the time array and each element of the `ABD` object; usually this is set using the system's total Christodoulou mass extracted from the metadata, which $\approx1.0$)
- `t_interpolate`: a time array to interpolate to, e.g., the time array of the worldtube file
- `t_0_superrest`: the time at which to fix the BMS frame to the superrest frame (we'll discuss this later in this session)
- `padding_time`: the window encompassing `t_0_superrest` to use when fixing the BMS frame (we'll discuss this later in this session)

So, without further ado, let's create an `ABD` object!

In [ ]:
reductions_file_name = Path("/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/CharacteristicExtractReduction.h5")
with h5py.File(reductions_file_name) as input_file:
    radius = [x for x in input_file.keys() if 'Spectre' in x][0].split('SpectreR')[1][:4]

worldtube_file_name = f"/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/BondiCceR{radius}.h5"
with h5py.File(worldtube_file_name) as input_file:
    t_worldtube = scri.SpEC.file_io.monotonize(
        np.array(input_file["J.dat"])[:, 0]
    )
# We'll use a very sparse time array to speed up computations
t_interpolate = np.arange(t_worldtube[0], t_worldtube[-1] + 1.0, 1.0)

abd = scri.create_abd_from_h5(
    file_format="SpECTRECCE_v1",
    file_name=str(reductions_file_name.resolve()),
    radius=radius,
    ch_mass=1.0, # this should be the system's total Christodoulou mass according to the metadata! We'll cover how to obtain this in the waveform data session.
    t_interpolate=t_interpolate
)

## Checking the Bianchi violations

One way to check that CCE worked and that our asymptotic bondi data is consistent with GR is to check the violation of the Bianchi identities:
- $\dot{\Psi}_{0}=\eth\Psi_{1}+3\sigma\Psi_{2}$
- $\dot{\Psi}_{1}=\eth\Psi_{2}+2\sigma\Psi_{2}$
- $\dot{\Psi}_{2}=\eth\Psi_{3}+\sigma\Psi_{4}$
- $\Psi_{3}=-\eth\dot{\bar{\sigma}}$
- $\Psi_{4}=-\ddot{\bar{\sigma}}$
- $\mathrm{Im}[\Psi_{2}]=-\mathrm{Im}[\eth^{2}\bar{\sigma}+\sigma\dot{\bar{\sigma}}]$

Dots denote time derivatives. Bars denote complex conjugates. $\eth$ is the spin-weight operator. When acting on a spin-weight $s$ function $f$ it yields

$\eth f\equiv-\frac{1}{\sqrt{2}}\left(\sin\theta\right)^{+s}\left(\partial_{\theta}+i\csc\theta\partial_{\phi}\right)\left[\left(\sin\theta\right)^{-s}f\right]$;

or when acting on a spin-weight spherical harmonic $\phantom{}_{s}Y_{(\ell,m)}$ it yields

$\eth\phantom{}_{s}Y_{(\ell,m)}=\frac{1}{\sqrt{2}}\sqrt{(\ell-s)(\ell+s+1)}\phantom{}_{s+1}Y_{(\ell,m)}$

To check these, one can run `abd.bondi_violation_norm`. Let's try it!.

In [ ]:
bianchi_violations = abd.bondi_violation_norms

And now let's plot them!

In [ ]:
plt.semilogy(
    abd.t,
    np.array(bianchi_violations).T,
    label=['$\dot{\Psi}_{0}$','$\dot{\Psi}_{1}$','$\dot{\Psi}_{2}$','$\Psi_{3}$','$\Psi_{4}$','$\mathrm{Im}[\Psi_{2}]$']
)
plt.legend(ncol=2)
plt.xlabel('Time (M)')

Overall this looks pretty reasonable. Any ideas for why $\Psi_{0}$ and $\Psi_{1}$ exhibit larger violations?

## Plotting Waveforms

Alright, now let's look at some waveforms! Let's start with the shear $\sigma$ (which, as a reminder, is related to the strain $h$ via $h=2\bar{\sigma}$).

In [ ]:
plt.plot(
    abd.t,
    abs(abd.sigma.ndarray[:,sf.LM_index(2,-2,0):sf.LM_index(2,2,0) + 1]),
    label=['$(2,-2)$','$(2,-1)$','$(2,0)$','$(2,+1)$','$(2,+2)$']
)
plt.legend()
plt.xlabel('Time (M)')
plt.ylabel('shear')

hmm, this looks a little weird; this is a $q=1$, non-spinning, quasi-circular BBH merger, so why is the shear's/strain's amplitude oscillatory?

Maybe we can look at the charges to see if we notice anything strange.

## Examining Oddities in the Data

Let's start with the center-of-mass charge. After all, we have yet to perform a center-of-mass correction!

In [ ]:
G = abd.bondi_CoM_charge()

In [ ]:
plt.plot(abd.t, G, label=['$G_{x}$','$G_{y}$','$G_{z}$'])
plt.legend()
plt.ylabel('center-of-mass charge')

Uh oh! The above plot corresponds to our system's center-of-mass position; so it looks like our system's center-of-mass is moving! 

Let's fix that.

First we compute the transformation to map to the center-of-mass frame, i.e., the rest frame of the system.

In [ ]:
t_0 = 2000
padding_time = 200
idx1 = np.argmin(abs(abd.t - (t_0 - padding_time)))
idx2 = np.argmin(abs(abd.t - (t_0 + padding_time))) + 1
abd_inspiral = abd.interpolate(abd.t[idx1:idx2]) 

CoM_transformation, _ = scri.asymptotic_bondi_data.map_to_superrest_frame.com_transformation_to_map_to_superrest_frame(
    abd_inspiral
)

This function returns a `BMSTransformation` object:

In [ ]:
CoM_transformation

These objects store a BMS transformation by expressing it in terms of a supertranslation, rotation, and boost, with some specified order.

They can be
- reordered via `BMSTransformation.reorder()`,
- composed via `BMSTransformation1 * BMSTransformation2`, 
- and even inverted via `BMSTransformation.inverse()`.

For more on how BMS transformations act on asymptotic data, see [https://arxiv.org/abs/1509.00862](arXiv:1509.00862).

With this BMS transformation, we can transform our `ABD` object using the `ABD.transform` function. 

In [ ]:
abd_rest = abd.transform(
    supertranslation=CoM_transformation.supertranslation,
    frame_rotation=CoM_transformation.frame_rotation.components,
    boost_velocity=CoM_transformation.boost_velocity
)

In [ ]:
G_rest = abd_rest.bondi_CoM_charge()

In [ ]:
plt.plot(abd_rest.t, G_rest, label=['$G_{x}$','$G_{y}$','$G_{z}$'])
plt.legend()
plt.ylabel('center-of-mass charge')

Much better! Did that fix the modes of our waveform?

In [ ]:
plt.plot(
    abd_rest.t,
    abs(abd_rest.sigma.ndarray[:,sf.LM_index(2,-2,0):sf.LM_index(2,2,0) + 1]),
    label=['$(2,-2)$','$(2,-1)$','$(2,0)$','$(2,+1)$','$(2,+2)$']
)
plt.legend()
plt.xlabel('Time (M)')
plt.ylabel('shear')

Grr. Doesn't look like it.

Maybe there's something wrong with the supertranslation charge? (since the rotation charge is trivial for non-spinning systems)

In [ ]:
Moreschi_supermomentum = abd_rest.supermomentum('Moreschi')

In [ ]:
plt.plot(
    abd_rest.t,
    Moreschi_supermomentum.ndarray[:,sf.LM_index(2,-2,0):sf.LM_index(2,2,0)+1].real,
    label=['$(2,-2)$','$(2,-1)$','$(2,0)$','$(2,+1)$','$(2,+2)$']
)
plt.legend()
plt.xlabel('Time (M)')
plt.ylabel('Moreschi supermomentum')

Oh! Look at that! The $(2,\pm2)$ modes of the Moreschi supermomentum seem to be some nonzero, constant value, which suggests that our early time spacetime won't asymptote to the usual superimposed Kerr metric, but rather a _supertranslated_ superimposed Kerr metric.

Let's see if mapping them to zero fixes things. This will take ~3 minutes.

This frame that we're mapping to is called the _superrest frame_.

In [ ]:
abd_superrest, superrest_BMS_transformation, _ = abd.map_to_superrest_frame(
    t_0=t_0,
    padding_time=padding_time
)

In [ ]:
Moreschi_supermomentum_superrest = abd_superrest.supermomentum('Moreschi')

In [ ]:
plt.plot(
    abd_superrest.t,
    Moreschi_supermomentum_superrest.ndarray[:,sf.LM_index(2,-2,0):sf.LM_index(2,2,0)+1],
    label=['$(2,-2)$','$(2,-1)$','$(2,0)$','$(2,+1)$','$(2,+2)$']
)
plt.legend()
plt.xlabel('Time (M)')
plt.ylabel('Moreschi supermomentum')

Much better! But did this fix our problem?

In [ ]:
plt.plot(
    abd_superrest.t,
    abs(abd_superrest.sigma.ndarray[:,sf.LM_index(2,-2,0):sf.LM_index(2,2,0) + 1]),
    label=['$(2,-2)$','$(2,-1)$','$(2,0)$','$(2,+1)$','$(2,+2)$']
)
plt.legend()
plt.xlabel('Time (M)')
plt.ylabel('shear')

Wonderful!

Any ideas what the early time oscillations are?

We can now proceed to study our waveforms as we please.

If you want to look at the Weyl scalars, simply replace `sigma` by `psi0`, `psi1`, `psi2`, `psi3`, or `psi4`.

## BMS Charge Decomposition of a Gravitational Wave

One nice way to study NR data, now that we have the Weyl scalars, is through the BMS charge decomposition of the shear:
- $\mathrm{Re}\left[\bar{\eth}^{2}\sigma\right]=\mathrm{Re}\left[m+\mathcal{E}+\left(\bar{\eth}^{2}\sigma - m\right)|^{u_{0}}\right]$
- $\mathrm{im}\left[\bar{\eth}^{2}\sigma\right]=-\frac{d}{du}\left(\eth\bar{\eth}\right)^{-1}\mathrm{Im}\left[\bar{\eth}\left(\hat{N}+\mathcal{J}\right)\right]$

Let's compute these terms.

In [ ]:
re_eth_sq_sigma = abd_superrest.sigma.ethbar_GHP.ethbar_GHP.real
mass_aspect = -(abd_superrest.psi2 + abd_superrest.sigma.multiply(abd_superrest.sigma.dot.bar, truncator=lambda tup: 8)).real
energy_flux = abd_superrest.sigma.dot.multiply(abd_superrest.sigma.dot.bar, truncator=lambda tup: 8).int.real

idx = np.argmin(abs(abd_superrest.t - t_0))
energy_flux -= (energy_flux[idx] - (re_eth_sq_sigma[idx] - mass_aspect[idx]))

In [ ]:
def apply_eth_ethbar_inverse(f):
    s = f.spin_weight
    for L in range(int(np.sqrt(f.ndarray.shape[1])) - 1 + 1):
        for M in range(-L, L + 1):
            if L < 2:
                f.ndarray[:,sf.LM_index(L, M, 0)] *= 0
            else:
                f.ndarray[:,sf.LM_index(L, M, 0)] *= ((-1/np.sqrt(2)*np.sqrt((L + s)*(L - s + 1))) * (1/np.sqrt(2)*np.sqrt((L - (s - 1))*(L + (s - 1) + 1))))**(-1)
            
    return f                                                                             

In [ ]:
im_eth_sq_sigma = -abd_superrest.sigma.ethbar_GHP.ethbar_GHP.imag
Lorentz_aspect = apply_eth_ethbar_inverse(-(abd_superrest.psi1 + abd_superrest.sigma.multiply(abd_superrest.sigma.bar.eth_GHP, truncator=lambda tup: 8)).dot.ethbar_GHP).imag
Lorentz_flux = apply_eth_ethbar_inverse(0.5*(3*abd_superrest.sigma.dot.multiply(abd_superrest.sigma.bar.eth_GHP, truncator=lambda tup: 8) - 3*abd_superrest.sigma.multiply(abd_superrest.sigma.dot.bar.eth_GHP, truncator=lambda tup: 8) + abd_superrest.sigma.bar.multiply(abd_superrest.sigma.dot.eth_GHP, truncator=lambda tup: 8) - abd_superrest.sigma.dot.bar.multiply(abd_superrest.sigma.eth_GHP, truncator=lambda tup: 8)).ethbar_GHP).imag

If we plot some of the more interesting modes, we find

In [ ]:
fig, axis = plt.subplots(2, 2, sharex=True)
plt.subplots_adjust(hspace=0.08, wspace=0.08)
fig.align_ylabels()

idx1 = np.argmin(abs(abd_superrest.t - 5800))
idx2 = np.argmin(abs(abd_superrest.t - 6080))

mode = (2,2)
axis[0][0].plot(abd_superrest.t[idx1:idx2], re_eth_sq_sigma[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], label='$\sigma$')
axis[0][0].plot(abd_superrest.t[idx1:idx2], mass_aspect[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], ls='--', label='$m$')
axis[0][0].plot(abd_superrest.t[idx1:idx2], energy_flux[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], ls='dotted', label='$\mathcal{E}$')

mode = (2,0)
axis[1][0].plot(abd_superrest.t[idx1:idx2], re_eth_sq_sigma[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], label='$\sigma$')
axis[1][0].plot(abd_superrest.t[idx1:idx2], mass_aspect[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], ls='--', label='$m$')
axis[1][0].plot(abd_superrest.t[idx1:idx2], energy_flux[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], ls='dotted', label='$\mathcal{E}$')

mode = (3,2)
axis[0][1].plot(abd_superrest.t[idx1:idx2], im_eth_sq_sigma[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], label='$\sigma$')
axis[0][1].plot(abd_superrest.t[idx1:idx2], Lorentz_aspect[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], ls='--', label='$\hat{N}$')
axis[0][1].plot(abd_superrest.t[idx1:idx2], Lorentz_flux[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], label='$\mathcal{J}$')

mode = (3,0)
axis[1][1].plot(abd_superrest.t[idx1:idx2], im_eth_sq_sigma[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], label='$\sigma$')
axis[1][1].plot(abd_superrest.t[idx1:idx2], Lorentz_aspect[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], ls='--', label='$\hat{N}$')
axis[1][1].plot(abd_superrest.t[idx1:idx2], Lorentz_flux[idx1:idx2,sf.LM_index(mode[0], mode[1], 0)], label='$\mathcal{J}$')

axis[0][0].legend(loc='upper left')
axis[0][1].legend(loc='upper left')
axis[1][0].legend(loc='upper left')
axis[1][1].legend(loc='upper left')

axis[0][1].yaxis.set_label_position("right")
axis[0][1].yaxis.tick_right()

axis[1][1].yaxis.set_label_position("right")
axis[1][1].yaxis.tick_right()

axis[1][0].set_xlabel('Time (M)')
axis[1][1].set_xlabel('Time (M)')

axis[0][0].set_ylabel(r'Re of $(2,2)$ mode')
axis[0][1].set_ylabel(r'Re of $(3,2)$ mode', rotation=270, labelpad=14)
axis[1][0].set_ylabel(r'Re of $(2,0)$ mode')
axis[1][1].set_ylabel(r'Im of $(3,0)$ mode', rotation=270, labelpad=14)

Wow! Look at that!
- For the $(2,2)$ mode the strain is entirely sourced by the mass aspect; this is just the mass quadrupole formula!
- For the $(2,0)$ mode the energy flux sources the memory contribution to the strain and the mass aspect captures ringdown excitations;
- For the $(3,2)$ mode the strain is entirely sourced by the Lorenz aspect; this is the lesser known current multipole formula;
- For the $(3,0)$ mode the Lorentz flux sources the "bump" contribution to the strain; this is the spin memory effect!

## Working with Ringdown Data

Finally, let's imagine we wanted to study the ringdown phase of our BBH merger to, say, work towards performing a test of GR with the remnant black hole.

To study the ringdown, we want our remnant black hole to asymptote to the Kerr metric.

But look! Our remnant doesn't asymptote to the Kerr metric because the supermomentum is nontrivial!

In [ ]:
plt.plot(
    abd_superrest.t,
    Moreschi_supermomentum_superrest.ndarray[:,sf.LM_index(2,-2,0):sf.LM_index(2,2,0)+1],
    label=['$(2,-2)$','$(2,-1)$','$(2,0)$','$(2,+1)$','$(2,+2)$']
)
plt.xlim(5000)
plt.legend()
plt.xlabel('Time (M)')
plt.ylabel('Moreschi supermomentum')

At late times, our remnant is _supertranslated_ Kerr, with a large supertranslation in the $(2,0)$ mode due to the memory sourced by the merger.

Let's fix this.

In [ ]:
t_0 = 6400
padding_time = 50

abd_remnant_superrest, remnant_superrest_BMS_transformation, _ = abd.map_to_superrest_frame(
    t_0=t_0,
    padding_time=padding_time
)

In [ ]:
Moreschi_supermomentum_remnant_superrest = abd_remnant_superrest.supermomentum('Moreschi')

In [ ]:
plt.plot(
    abd_remnant_superrest.t,
    Moreschi_supermomentum_remnant_superrest.ndarray[:,sf.LM_index(2,-2,0):sf.LM_index(2,2,0)+1],
    label=['$(2,-2)$','$(2,-1)$','$(2,0)$','$(2,+1)$','$(2,+2)$']
)
plt.xlim(5000)
plt.legend()
plt.xlabel('Time (M)')
plt.ylabel('Moreschi supermomentum')

Much better!

And finally, one might want to know what the mass and spin of our remnant black hole are.

To obtain these, we can just use the BMS charges! (see [https://arxiv.org/abs/2104.07052](arXiv:2104.07052))

In [ ]:
remnant_mass = abd_remnant_superrest.bondi_rest_mass()[-1]
remnant_spin = abd_remnant_superrest.bondi_dimensionless_spin()[-1]
print(f'Mass: {remnant_mass}')
print(f'Dimensionless spin: {remnant_spin}')

Cool! With this information, we have everything that we need to fit black hole perturbation theory excitations, like quasi-normal modes (QNMs), to the ringdown phase of the waveforms emitted by our system!

## Mapping to the PN BMS frame

Apart from mapping to a BMS frame that is ideal for studying the ringdown phase of our system, we can also map to a BMS frame to compare with waveform models, like post-Newtonian waveforms! For this there are two options:
- Optimize over the BMS transformation to minimize the residual between the two waveforms
- Find the transformation that maps one set of charges to the other

These options were explored in [https://arxiv.org/abs/2105.02300](arXiv:2105.02300) and [https://arxiv.org/abs/2208.04356](arXiv:2208.04356). Naturally, the 2nd is much faster! But to do so requires that we have _target_ charges. So let's load some PN waveforms!

This is the strain

In [ ]:
h_PN = scri.SpEC.file_io.read_from_h5('/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/rhOverM_PN_T4.h5')

and this is the Moreschi supermomentum

In [ ]:
PsiM_PN = scri.SpEC.file_io.read_from_h5('/oscar/data/icerm/knelli/workshop_materials/03_wednesday/cce/supermomentum_Moreschi_PN_T4.h5')

As you may expect, the agreement between PN and NR out of the box isn't so nice...

In [ ]:
fig, axis = plt.subplots(2, 1, sharex=True)
plt.subplots_adjust(hspace=0.05)

axis[0].plot(abd.t, 2.0*abd.sigma.bar.ndarray[:,sf.LM_index(2,2,0)], label='NR')
axis[0].plot(h_PN.t, h_PN.data[:,sf.LM_index(2,2,h_PN.ell_min)], ls='--', label='PN')

axis[1].plot(abd.t, 2.0*abd.sigma.bar.ndarray[:,sf.LM_index(2,0,0)], label='NR')
axis[1].plot(h_PN.t, h_PN.data[:,sf.LM_index(2,0,h_PN.ell_min)], ls='--', label='PN')

axis[1].set_xlabel('Time (M)')
axis[0].set_ylabel('strain (2,2) mode')
axis[1].set_ylabel('strain (2,0) mode')
axis[0].legend()
axis[1].legend()

But this is simply because we need to map NR to the frame of the PN! So let's fix some frame freedoms.

We'll fix the rotation freedom with the angular velocity vector (see [https://arxiv.org/abs/1302.2919](arXiv:1302.2919) for more details) and the center-of-mass freedom by mapping the center-of-mass charge to zero. The supertranslation freedom will be fixed by the PN Moreschi supermomentum (see [https://arxiv.org/abs/2208.04356](arXiv:2208.04356) for how we compute this).

This can again be performed using the `map_to_superrest_function`, but now with the following options set:
- `target_strain_input`: the target PN strain
- `target_PsiM_input`: the target PN Moreschi supermomentum

In [ ]:
abd_PN_BMS, PN_BMS_transformation, _ = abd.map_to_superrest_frame(
    t_0=2000,
    padding_time=250,
    target_strain_input=h_PN,
    target_PsiM_input=PsiM_PN
)

How'd we do?

In [ ]:
fig, axis = plt.subplots(2, 1, sharex=True)
plt.subplots_adjust(hspace=0.05)

axis[0].plot(abd_PN_BMS.t, 2.0*abd_PN_BMS.sigma.bar.ndarray[:,sf.LM_index(2,2,0)], label='NR')
axis[0].plot(h_PN.t, h_PN.data[:,sf.LM_index(2,2,h_PN.ell_min)], ls='--', label='PN')

axis[1].plot(abd_PN_BMS.t, 2.0*abd_PN_BMS.sigma.bar.ndarray[:,sf.LM_index(2,0,0)], label='NR')
axis[1].plot(h_PN.t, h_PN.data[:,sf.LM_index(2,0,h_PN.ell_min)], ls='--', label='PN')

axis[1].set_xlabel('Time (M)')
axis[0].set_ylabel('strain (2,2) mode')
axis[1].set_ylabel('strain (2,0) mode')
axis[0].legend()
axis[1].legend()

axis[0].axvline(2000 - 250)
axis[0].axvline(2000 + 250)

axis[1].axvline(2000 - 250)
axis[1].axvline(2000 + 250)

Not bad! Thanks BMS!

## Mapping to the BMS frame of another ABD object

Finally, one can also map to the BMS frame of another ABD object by again finding the BMS transformation that maps the BMS charges of one system to the other. This is important for, e.g., performing convergence tests (since different Levs will likely be output in different BMS frames!). To do this, we use the `map_to_abd_frame` function.

But, because we don't have a different Lev right now, we'll just map our abd to some arbitrary BMS frame using the PN BMS transformation and see if we can recover using this function.

In [ ]:
abd_sparser = abd.interpolate(np.arange(abd.t[0], abd.t[-1] + 5, 5))

In [ ]:
abd_random_frame = abd_sparser.transform(
    supertranslation=PN_BMS_transformation.supertranslation,
    frame_rotation=PN_BMS_transformation.frame_rotation.components,
    boost_velocity=PN_BMS_transformation.boost_velocity
)

In [ ]:
fig, axis = plt.subplots(2, 1, sharex=True)
plt.subplots_adjust(hspace=0.05)

axis[0].plot(abd_sparser.t, 2.0*abd_sparser.sigma.bar.ndarray[:,sf.LM_index(2,2,0)], label='NR')
axis[0].plot(abd_random_frame.t, 2.0*abd_random_frame.sigma.bar.ndarray[:,sf.LM_index(2,2,0)], ls='--', label='NR (random frame)')

axis[1].plot(abd_sparser.t, 2.0*abd_sparser.sigma.bar.ndarray[:,sf.LM_index(2,0,0)], label='NR')
axis[1].plot(abd_random_frame.t, 2.0*abd_random_frame.sigma.bar.ndarray[:,sf.LM_index(2,0,0)], ls='--', label='NR (random frame)')

axis[1].set_xlabel('Time (M)')
axis[0].set_ylabel('strain (2,2) mode')
axis[1].set_ylabel('strain (2,0) mode')
axis[0].legend()
axis[1].legend()

axis[0].axvline(2000 - 250)
axis[0].axvline(2000 + 250)

axis[1].axvline(2000 - 250)
axis[1].axvline(2000 + 250)

Clearly they're in different frames! Let's try to fix this. (This will take a bit longer...)

In [ ]:
abd_to_random_frame, BMS_transformation_to_random_frame, _ = abd_sparser.map_to_abd_frame(
    abd_random_frame,
    t_0=2500,
    padding_time=500,
    N_itr_maxes={"abd": 2, "superrest": 1, "CoM_transformation": 10, "rotation": 10, "supertranslation": 10},
)

In [ ]:
fig, axis = plt.subplots(2, 1, sharex=True)
plt.subplots_adjust(hspace=0.05)

axis[0].plot(abd_to_random_frame.t, 2.0*abd_to_random_frame.sigma.bar.ndarray[:,sf.LM_index(2,2,0)], label='NR (to random frame)')
axis[0].plot(abd_random_frame.t, 2.0*abd_random_frame.sigma.bar.ndarray[:,sf.LM_index(2,2,0)], ls='--', label='NR (random frame)')

axis[1].plot(abd_to_random_frame.t, 2.0*abd_to_random_frame.sigma.bar.ndarray[:,sf.LM_index(2,0,0)], label='NR (to random frame)')
axis[1].plot(abd_random_frame.t, 2.0*abd_random_frame.sigma.bar.ndarray[:,sf.LM_index(2,0,0)], ls='--', label='NR (random frame)')

axis[1].set_xlabel('Time (M)')
axis[0].set_ylabel('strain (2,2) mode')
axis[1].set_ylabel('strain (2,0) mode')
axis[0].legend()
axis[1].legend()

axis[0].axvline(2000 - 250)
axis[0].axvline(2000 + 250)

axis[1].axvline(2000 - 250)
axis[1].axvline(2000 + 250)

Much better!

And, for the cherry on top, if we compose the BMS transformations, do we obtain the identity transformation?

In [ ]:
BMS_transformation_to_random_frame

In [ ]:
BMS_transformation_to_random_frame * PN_BMS_transformation.inverse()

Indeed! (or at least pretty close!) Isn't GR amazing???